## Import the data

In [ ]:
# Import mosaic libraries
import missionbio.mosaic as ms

# Import these to display entire dataframes
from IPython.display import display, HTML

# Import graph_objects from the plotly package to display figures when saving the notebook as an HTML
import plotly as px
import plotly.graph_objects as go

# Import additional packages for specific visuals
import matplotlib.pyplot as plt
import plotly.offline as pyo
pyo.init_notebook_mode()
import numpy as np
import seaborn as sns
from missionbio.plotting.multimap import MultiMap
from missionbio.plotting.heatmap import Heatmap

# Import COMPASS for imputation
from missionbio.mosaic.algorithms.compass import COMPASS
import pandas as pd

In [4]:
# This code is optional, but will make the notebook cells/figures display across the entire width of the browser
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

Read the results of WGS experiment from the excel file.

In [5]:
file_xlxs = "/orfeo/LTS/CDSLab/LT_storage/CLL/scDNA/h5_file/Tapestry CLL Targeted Panel.xlsx"
dfs = pd.read_excel(file_xlxs, sheet_name=None)

In [6]:
wgs_muts = dfs["X_Tapestry"]

Select only the mutations specific for the sample of interest. Take only the mutations which underwent Tapsetri analysis and have a CCF value.

In [ ]:
patient_id = "CT339"
sample_id = "T2_pos"
cff_name = "CCF_"+sample_id
CT339_muts = wgs_muts[(wgs_muts["Case"]==patient_id) & (wgs_muts["TAPESTRI"]=="YES") & (wgs_muts[cff_name]!=0)]
CT339_muts["Tapestri_id"] =CT339_muts[['Chr','Start','Ref','Var']].apply(lambda x : '{}:{}:{}/{}'.format(x[0],x[1],x[2],x[3]), axis=1)
whitelist = list(CT339_muts[['Chr','Start','Ref','Var']].apply(lambda x : '{}:{}:{}/{}'.format(x[0],x[1],x[2],x[3]), axis=1))

Load the data from the h5 file. IN this case I decided to extract only the variants that are present in the previoulsy generated whitelist and to not apply the fitlering procedure.

In [8]:
h5path = "/orfeo/LTS/CDSLab/LT_storage/CLL/scDNA/h5_file/CT339_T2_POS_Test_11.dna.h5"
# Load the data
sample = ms.load(h5path, raw=False, filter_variants=False, single=True, whitelist = whitelist, filter_cells=False) 

            0%: 

/u/cdslab/ggandolfi/mosaic/lib/python3.9/site-packages/missionbio/h5/data/reader.py:593: UserWarning:

The following were not found: {'chr2:132794309:C/T', 'chr14:77883003:C/T', 'chr10:23361627:C/T', 'chr8:37454107:A/C', 'chr1:148002257:C/T', 'chr1:59096386:C/G', 'chr4:65299766:A/C', 'chr7:142475932:C/G'}



### Inspect the data structure

In [21]:
# Summary of DNA assay 
print("\'sample.dna\':", sample.dna, '\n')
print("\'row_attrs\':", "\n\t", list(sample.dna.row_attrs.keys()), '\n')
print("\'col_attrs\':", "\n\t", list(sample.dna.col_attrs.keys()), '\n')
print("\'layers\':", "\n\t", list(sample.dna.layers.keys()), '\n')
print("\'metadata\':", "\n")
#for i in list(sample.dna.metadata.keys()):
#    print("\t", i, ": ", sample.cnv.metadata[i], sep="")

'sample.dna': dna_variants assay with 9 layers, 4705 rows and 38 columns 

'row_attrs': 
	 ['barcode', 'filtered', 'sample_name', 'label'] 

'col_attrs': 
	 ['ALT', 'CHROM', 'POS', 'QUAL', 'REF', 'ado_gt_cells', 'ado_rate', 'amplicon', 'filtered', 'id'] 

'layers': 
	 ['AF', 'DP', 'FILTER_MASK', 'GQ', 'NGT', 'RGQ', 'AF_MISSING', 'NGT_FILTERED', 'AF_FILTERED'] 

'metadata': 



In [22]:
# Summary of CNV assay
print("\'sample.cnv\':", sample.cnv, '\n')
print("\'row_attrs\':", "\n\t", list(sample.cnv.row_attrs.keys()), '\n')
print("\'col_attrs\':", "\n\t", list(sample.cnv.col_attrs.keys()), '\n')
print("\'layers\':", "\n\t", list(sample.cnv.layers.keys()), '\n')
print("\'metadata\':", "\n")
for i in list(sample.cnv.metadata.keys()):
    print("\t", i, ": ", sample.cnv.metadata[i], sep="")

'sample.cnv': dna_read_counts assay with 1 layer, 4705 rows and 489 columns 

'row_attrs': 
	 ['barcode', 'sample_name', 'label'] 

'col_attrs': 
	 ['CHROM', 'end_pos', 'id', 'r1_counts', 'r2_counts', 'start_pos'] 

'layers': 
	 ['read_counts'] 

'metadata': 

	sample_name: [['Test_11']]
	ado_rate: 0.134
	avg_mapping_error_rate: 0.006985091
	avg_panel_uniformity: 0.8466257668711656
	chemistry_version: V2
	genome_version: hg19
	n_amplicons: 489
	n_bases_r1: 108421897969
	n_bases_r1_q30: 99960596626
	n_bases_r2: 108452686498
	n_bases_r2_q30: 98507214574
	n_cell_barcode_bases: 36401460383
	n_cell_barcode_bases_q30: 35637056782
	n_cells: 4705
	n_read_pairs: 723945695
	n_read_pairs_mapped_to_cells: 449219436
	n_read_pairs_trimmed: 718587018
	n_read_pairs_valid_cell_barcodes: 699883460
	n_reads_mapped: 1342502928
	n_reads_mapped_insert: 1259460099
	panel_name: Tapestri-Designer-results-6351
	pipeline_version: 2.0.2


Apply the filters to the input whitelisted variants.

In [24]:
# Adjust filters if needed by overwriting dna_vars
dna_vars = sample.dna.filter_variants(
    min_dp=10,
    min_gq=30,
    vaf_ref=5,
    vaf_hom=95,
    vaf_het=30,
    min_prct_cells=50,
    min_mut_prct_cells=1,
    iterations=10,
)

# Check the number of filtered variants. When using the default filters, the number of 
# variants is likely smaller compared to the originally loaded variants due to the more 
# stringent filtering criteria (e.g., vaf_ref=5, vaf_hom=95, vaf_het=30).
print('Number of variants:', len(dna_vars))
#print(dna_vars)

Number of variants: 28


In [25]:
final_vars = list(set(list(dna_vars) + whitelist))
sample.dna = sample.dna[:, final_vars]
sample.dna.shape

(4705, 38)

Effectively apply the fitlering:

Create a dataframe with the variants information. This dataframe contains information about:
- `ALT`: The alternate allele for each variant;
- `CHROM`
- `POS`
- `QUAL`
- `REF`
- `ado_gt_cells`
- `ado_rate`
- `amplicon`
- `filtered`
- `id`

In [19]:
filtered_variants = list(sample.dna.filter_variants())
#ann = sample.dna.get_annotations()
#whitelist

In [26]:
col_attributes = pd.DataFrame(sample.dna.col_attrs)
#pd.DataFrame(sample.dna.row_attrs)
#sample.dna.col_attrs.keys()

## Compare WGS bulk and Tapestri results

In [28]:
wgs_vs_tapestri = pd.DataFrame(whitelist, columns=['id'])
wgs_vs_tapestri["CCF_Tapestri"] = 0
wgs_vs_tapestri["CCF_WGS"] = 0
wgs_vs_tapestri["VAF_WGS"] = 0

In [31]:
af = pd.DataFrame(sample.dna.layers["AF"])
af.columns = sample.dna.col_attrs["id"]
af.index = sample.dna.row_attrs["barcode"]

for v in whitelist:
    if (v in sample.dna.col_attrs["id"]):
        m = af[v]/ 100
        i=wgs_vs_tapestri.index[wgs_vs_tapestri['id'] == v]
        ccf =(m != 0).sum()/len(sample.dna.row_attrs["barcode"])
        wgs_vs_tapestri.loc[i, 'CCF_Tapestri'] =ccf
        ccf_wgs = CT339_muts[(CT339_muts["Tapestri_id"]==v)]
        wgs_vs_tapestri.loc[i, 'CCF_WGS'] =ccf_wgs["CCF_T2_pos"].iat[0]
        wgs_vs_tapestri.loc[i, 'VAF_WGS'] =ccf_wgs["VAF_T2_pos"].iat[0]
        #print(str(v)+" " +str(ccf)+" "+str(ccf_wgs["CCF_T2_pos"])+" "+str(ccf_wgs["VAF_T2_pos"]))
#m = af["chr14:47812259:G/A"]/ 100

#m[(m > 0.1)].hist(color='skyblue', edgecolor='black')
#(m != 0).sum()/len(sample.dna.row_attrs["barcode"])

In [32]:
wgs_vs_tapestri

,id,CCF_Tapestri,CCF_WGS,VAF_WGS
0,chr1:149035493:G/T,0.186610,0.119764,0.061856
1,chr6:353034:A/T,0.758767,0.119764,0.084444
2,chr4:65299766:A/C,0.000000,0.000000,0.000000
3,chr7:142475930:A/G,0.195962,0.119764,0.047120
4,chr7:142475932:C/G,0.000000,0.000000,0.000000
5,chr9:141069114:G/A,0.041233,0.119764,0.061905
6,chr10:696064:G/A,0.209777,0.119764,0.045113
7,chr1:148002257:C/T,0.000000,0.000000,0.000000
8,chr1:59096386:C/G,0.000000,0.000000,0.000000
9,chr11:1017381:G/C,0.882040,0.119764,0.106762


## DNA analysis

In [ ]:
# Run the variant table workflow to select variants and begin clone identification
wfv = ms.workflows.VariantSubcloneTable(sample)
wfv.run()
# The width can be adjusted if needed for long variant names
#wfv.run(width=3000)

In [ ]:
variants = wfv.selected_variants
variants

## CNA analysis

In [ ]:
sample.cnv.get_gene_names()

In [ ]:
# CNV workflow to filter, normalize and estimate ploidy
wfc = ms.workflows.CopyNumber(sample)
wfc.run()